In [ ]:
%pip install llama-index
%pip install llama-index-llms-huggingface
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-huggingface-api
%pip install llama-index-llms-ollama 

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [4]:
from llama_index.llms.ollama import Ollama 
llm = Ollama(model="llama3.1",request_timeout=120)

In [5]:
from llama_index.core import Settings
Settings.llm = llm
Settings.embed_model = embed_model 

In [ ]:
!mkdir data
!wget "https://www.dropbox.com/scl/fi/t1soxfjdp0v44an6sdymd/drake_kendrick_beef.pdf?rlkey=u9546ymb7fj8lk2v64r6p5r5k&st=wjzzrgil&dl=1" -O data/drake_kendrick_beef.pdf
!wget "https://www.dropbox.com/scl/fi/nts3n64s6kymner2jppd6/drake.pdf?rlkey=hksirpqwzlzqoejn55zemk6ld&st=mohyfyh4&dl=1" -O data/drake.pdf
!wget "https://www.dropbox.com/scl/fi/8ax2vnoebhmy44bes2n1d/kendrick.pdf?rlkey=fhxvn94t5amdqcv9vshifd3hj&st=dxdtytn6&dl=1" -O data/kendrick.pdf


In [ ]:
from llama_index.core import SimpleDirectoryReader

docs_kendrick = SimpleDirectoryReader(input_files=["data/kendrick.pdf"]).load_data()
docs_drake = SimpleDirectoryReader(input_files=["data/drake.pdf"]).load_data()
docs_both = SimpleDirectoryReader(input_files=["data/drake_kendrick_beef.pdf"]).load_data()

In [ ]:
response = llm.complete("do you like drake or kendrick lamar")
print(response)

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader

docs_both = SimpleDirectoryReader(input_files=["data/drake_kendrick_beef.pdf"]).load_data()

index = VectorStoreIndex.from_documents(docs_both)
query_engine = index.as_query_engine(similarity_top_k = 3)

response = query_engine.query("Tell me about family matters")

print(str(response))

In [ ]:
from llama_index.core import SummaryIndex
from llama_index.core import SimpleDirectoryReader

docs_both = SimpleDirectoryReader(input_files=["data/drake_kendrick_beef.pdf"]).load_data()

summary_index = SummaryIndex.from_documents(docs_both)
summary_engine = summary_index.as_query_engine()

In [ ]:
response = summary_engine.query("Given your accessment of this article , who won this beef ?")

print(str(response))

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts."
    ),
)

summary_tool = QueryEngineTool(
    index.as_query_engine(response_mode = "tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for something an entire document"
    )
)

In [ ]:
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool,summary_tool],select_multi=False,verbose=True
)

response = query_engine.query(
    "Tell me about the song meet the grahams - why is it significant"
)
print(response)

In [ ]:
!wget "https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip" -O "./data/chinook.zip"
!unzip "./data/chinook.zip"

In [ ]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column
)

engine = create_engine("sqlite:///chinook.nb")

In [ ]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(engine)

In [ ]:
from llama_index.core.indices.struct_store import NLSQLTableQueryEngine

query_engine =NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["tracks","artist"],
    llm=llm
)

In [ ]:
response = query_engine.query("tell me about all th tables")
print(response)

In [ ]:
from llama_index.llms.ollama import Ollama
from llama_index.core.prompts import PromptTemplate
from pydantic import BaseModel

class Restaurant(BaseModel):
    """A Restaurant with name , city and cuisine. """

    name: str
    city: str
    cuisine: str

llm = Ollama(model="llama3.1")
prompt_tmpl = PromptTemplate(
    "Generate a restaurant in a given city {city_name}"
)

In [ ]:
restaurant_obj = llm.structured_predict(
    Restaurant,prompt_tmpl,city_name="Miami"
)
print(restaurant_obj)

In [ ]:
from llama_index.core.memory import ChatMemoryBuffer
from llama_index.core.chat_engine import CondensePlusContextChatEngine

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

chat_engine = CondensePlusContextChatEngine.from_defaults(
    index.as_retriever(),
    memory=memory,
    llm=llm,
    context_prompt=(
         "You are a chatbot, able to have normal interactions, as well as talk"
        " about the Kendrick and Drake beef."
        "Here are the relevant documents for the context:\n"
        "{context_str}"
        "\nInstruction: Use the previous chat history, or the context above, to interact and help the user."
    ),
    verbose=True
)

In [ ]:
response = chat_engine.chat(
    "Tell me about the songs Drake released in the beef"
)
print(str(response))

In [ ]:
response = chat_engine.chat("What about Kendrick?")
print(str(response))

In [ ]:
import json
from typing import Sequence, List

from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.agent import ReActAgent

import nest_asyncio

nest_asyncio.apply()

In [ ]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two integers and returns the result integer"""
    return a - b


def divide(a: int, b: int) -> int:
    """Divides two integers and returns the result integer"""
    return a / b


multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)
subtract_tool = FunctionTool.from_defaults(fn=subtract)
divide_tool = FunctionTool.from_defaults(fn=divide)

In [ ]:
agent = ReActAgent.from_tools(
    [multiply_tool, add_tool, subtract_tool, divide_tool],
    llm=llm,
    verbose=True,
)

In [ ]:
response = agent.chat("What is (121 + 2) * 5?")
print(str(response))

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [ ]:
docs_both = SimpleDirectoryReader(input_files=["data/drake_kendrick_beef.pdf"]).load_data()

index = VectorStoreIndex.from_documents(docs_both)

docs_kendrick = SimpleDirectoryReader(input_files=["data/kendrick.pdf"]).load_data()
docs_drake = SimpleDirectoryReader(input_files=["data/drake.pdf"]).load_data()

drake_index = VectorStoreIndex.from_documents(docs_drake)
kendrick_index = VectorStoreIndex.from_documents(docs_kendrick)



query_engine = index.as_query_engine(similarity_top_k = 3)
drake_tool = QueryEngineTool(
    drake_index.as_query_engine(),
    metadata=ToolMetadata(
        name="drake_search",
        description="Useful for searching over Drake's life.",
    ),
)

kendrick_tool = QueryEngineTool(
    kendrick_index.as_query_engine(),
    metadata=ToolMetadata(
        name="kendrick_search",
        description="Useful for searching over Kendrick's life.",
    ),
)

query_engine_tools = [drake_tool, kendrick_tool]

In [ ]:
agent = ReActAgent.from_tools(
    query_engine_tools,  ## TODO: define query tools
    llm=llm,
    verbose=True,
)

In [ ]:
response = agent.chat("Tell me about how Kendrick and Drake grew up")
print(str(response))